In [1]:
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool

In [2]:
import os
from dotenv import load_dotenv
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint

load_dotenv()

huggingfacehub_access_token = os.getenv("HUGGINGFACEHUB_ACCESS_TOKEN")

if not huggingfacehub_access_token: 
    raise ValueError("❌ HUGGINGFACEHUB_ACCESS_TOKEN not found in environment variables.")


In [3]:
# Tool creation

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [4]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [5]:
multiply.name

'multiply'

In [6]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [7]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [8]:
# Tool binding
llm=HuggingFaceEndpoint(repo_id="openai/gpt-oss-120b",task="text-generation",huggingfacehub_api_token=huggingfacehub_access_token)

chat_model = ChatHuggingFace(
    llm=llm
)

In [9]:
chat_model.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 68, 'total_tokens': 99}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_6b919af976f774e48013', 'finish_reason': 'stop', 'logprobs': None}, id='run--58e52297-65f5-49eb-9fd4-70dba6ff5521-0', usage_metadata={'input_tokens': 68, 'output_tokens': 31, 'total_tokens': 99})

In [10]:
chat_model_with_tools = chat_model.bind_tools([multiply])

In [11]:
chat_model_with_tools.invoke("Hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 130, 'total_tokens': 159}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_6b919af976f774e48013', 'finish_reason': 'stop', 'logprobs': None}, id='run--9e81e480-75b8-4237-8e08-70361e20a09f-0', usage_metadata={'input_tokens': 130, 'output_tokens': 29, 'total_tokens': 159})

In [12]:
query = HumanMessage('can you multiply 3 with 1000')

In [13]:
messages = [query]

In [14]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [15]:
result = chat_model_with_tools.invoke(messages)

In [16]:
messages.append(result)

In [17]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{\n  "a": 3,\n  "b": 1000\n}', 'name': 'multiply', 'description': None}, 'id': '84c1647e5', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 138, 'total_tokens': 188}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_6b919af976f774e48013', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3c10b1f2-c4ca-4a97-8b4a-17b1c0d3b79e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '84c1647e5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 138, 'output_tokens': 50, 'total_tokens': 188})]

In [18]:
tool_result = multiply.invoke(result.tool_calls[0])

In [19]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='84c1647e5')

In [20]:
messages.append(tool_result)

In [21]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{\n  "a": 3,\n  "b": 1000\n}', 'name': 'multiply', 'description': None}, 'id': '84c1647e5', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 138, 'total_tokens': 188}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_6b919af976f774e48013', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3c10b1f2-c4ca-4a97-8b4a-17b1c0d3b79e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '84c1647e5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 138, 'output_tokens': 50, 'total_tokens': 188}),
 ToolMessage(content='3000', name='multiply', tool_call_id='84c1647e5')]

In [22]:
chat_model_with_tools.invoke(messages).content

'The product of\u202f3\u202fand\u202f1000\u202fis\u202f**3000**.'

In [23]:
# creating a currency converter tool
import requests
from typing import Annotated
from langchain_core.tools import InjectedToolArg

@tool
def get_conversion_factor(base_currency:str, target_currency:str)->float:
    """
    This tool takes in 2 currency codes and returns the exchange rate from base currency to target currency
    """
    url=f"https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}"
    
    try :
        response=requests.get(url)
        return response.json()
    except Exception as e:
        print("not fetched")


@tool
def convert_currency(base_currency_value:float, exchange_rate:Annotated[float, InjectedToolArg])->float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value
    """

    return base_currency_value * exchange_rate


In [24]:
convert_currency.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'number'},
 'exchange_rate': {'title': 'Exchange Rate', 'type': 'number'}}

In [25]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1757808001,
 'time_last_update_utc': 'Sun, 14 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1757894401,
 'time_next_update_utc': 'Mon, 15 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.327}

In [26]:
convert_currency.invoke({'base_currency_value':1000, 'exchange_rate':82.74})

82740.0

In [44]:
# tool binding

llm=HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b",task="text-generation",huggingfacehub_api_token=huggingfacehub_access_token)

chat_model = ChatHuggingFace(
    llm=llm
)

chat_model_with_tools=chat_model.bind_tools([get_conversion_factor,convert_currency])

In [45]:
messages = [HumanMessage("Convert 100 USD to INR")]

In [46]:
messages

[HumanMessage(content='Convert 100 USD to INR', additional_kwargs={}, response_metadata={})]

In [47]:
ai_message=chat_model_with_tools.invoke(messages)

In [48]:
messages.append(ai_message)

In [49]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'chatcmpl-tool-8c93131e85a14e81a5a7576d00155884',
  'type': 'tool_call'}]

In [50]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)

    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    print(conversion_rate)
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert_currency.invoke(tool_call)
    messages.append(tool_message2)



88.327


In [51]:
messages

[HumanMessage(content='Convert 100 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='{"base_currency":"USD","target_currency":"INR"}', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor', 'description': None}, 'id': 'chatcmpl-tool-8c93131e85a14e81a5a7576d00155884', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 470, 'prompt_tokens': 182, 'total_tokens': 652}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--08d9e64a-b775-406f-9b72-7c962aa6e14b-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': 'chatcmpl-tool-8c93131e85a14e81a5a7576d00155884', 'type': 'tool_call'}], usage_metadata={'input_tokens': 182, 'output_tokens': 470, 'total_tokens': 652}),
 ToolMessage(content='{"result": "success", "documentat

In [55]:
chat_model_with_tools.invoke(messages).content

'**Conversion Result**\n\n- **100\u202fUSD** × **88.327\u202fINR/USD** = **8,832.70\u202fINR**\n\nSo, 100\u202fUS\u202fdollars is roughly **₹8,832.70** Indian rupees.'

In [57]:
import re

def clean_response(text: str) -> str:
    # Remove LaTeX math
    text = re.sub(r"\\\(.*?\\\)", "", text)
    text = re.sub(r"\\\[.*?\\\]", "", text)
    # Replace bold markdown
    text = text.replace("**", "")
    # Replace newlines
    text = text.replace("\\n", "\n")
    return text.strip()

raw_output = chat_model_with_tools.invoke(messages).content


In [58]:
print(clean_response(raw_output))

100 USD is approximately ₹8,332.70 INR (using the current rate of 1 USD = 88.327 INR).
